In [1]:
#%run 'word2vec_data.ipynb'
import cPickle as pickle
%pylab inline

import scipy
import numpy as np
import pandas as pd
#import plotly.plotly as py
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords

from gensim import models


import math
import codecs
import re
import string
from time import time


## Sklearn
from sklearn import svm, datasets, preprocessing, metrics
import sklearn

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, precision_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.utils import shuffle


from sklearn.svm import SVC
from sklearn import datasets, feature_selection, cluster, feature_extraction, decomposition, metrics, model_selection
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from scipy import interp

#############################

#############################
### other stuff
from IPython import display

import os
#import ipyparallel as ipp
import warnings
warnings.filterwarnings('ignore')

# This is here to avoid all the Matplotlib warnings due to current bugs 
# - not a good idea to keep around
print ("Required Libraries loaded.")

Populating the interactive namespace from numpy and matplotlib
Required Libraries loaded.


C:\Users\CR107\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\CR107\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#!pip install --upgrade version_information
#%reload_ext version_information
%load_ext version_information 
%version_information scipy, numpy, sklearn, pandas, scipy, nltk, gensim, matplotlib, os, cpickle

Software,Version
Python,2.7.13 64bit [MSC v.1500 64 bit (AMD64)]
IPython,5.1.0
OS,Windows 7 6.1.7601 SP1
scipy,0.18.1
numpy,1.11.3
sklearn,0.18.1
pandas,0.19.2
scipy,0.18.1
nltk,3.2.2
gensim,1.0.1


In [3]:
inhibitor  = pd.read_csv("C:/EPC_Data/complete_data/raw/Triptans.csv", sep=",", index_col = "PMID")

In [4]:
seed = 29
# shuffle dataset and split to train and test
inhibitor = shuffle(inhibitor, random_state = seed)
inhibitor.head(2)

,study,EndNote_Id,init_screen,fin_screen,Abstract,MeSH,Mh_MeSH,Pt_Pub_Type,Pub_Type,TiAbs,TiAbsMeSHPT,TiAbsMeSHPT_append,TiAbsMeSH_append,TiAbsMesh,Title,Label
PMID,,,,,,,,,,,,,,,,
12383060,Triptans,2159,E,E,"The triptans, selective serotonin 5-HT1B/1D a...","Administration, Oral Clinical Trials as Topic...","[MHAdministration,, MHOral, MHClinical, MHTria...","[PTJournal, PTArticle, PTMeta-Analysis, PTRese...",Journal Article Meta-Analysis Research Suppor...,"Triptans (serotonin, 5-HT1B/1D agonists) in mi...","Triptans (serotonin, 5-HT1B/1D agonists) in mi...","Triptans (serotonin, 5-HT1B/1D agonists) in mi...","Triptans (serotonin, 5-HT1B/1D agonists) in mi...","Triptans (serotonin, 5-HT1B/1D agonists) in mi...","Triptans (serotonin, 5-HT1B/1D agonists) in mi...",0
11723269,Triptans,129,I,E,To assess the efficacy and tolerability of or...,"Acute Disease Administration, Oral Adult Dose...","[MHAcute, MHDisease, MHAdministration,, MHOral...","[PTClinical, PTTrial, PTClinical, PTTrial,, PT...","Clinical Trial Clinical Trial, Phase II Compa...","Dose finding, placebo-controlled study of oral...","Dose finding, placebo-controlled study of oral...","Dose finding, placebo-controlled study of oral...","Dose finding, placebo-controlled study of oral...","Dose finding, placebo-controlled study of oral...","Dose finding, placebo-controlled study of oral...",0


In [5]:
folder = "C:/EPC_Data/complete_data/word2vec"
filepath = os.path.join(folder, "Triptans_chi2_features_10minwords_15context")
model = models.Word2Vec.load(filepath)
print (model["clinical"][:10])
print ("\nMatrix of word vectors is size : ", model.wv.syn0.shape)

[ 0.05524512  0.04734685 -0.16632569 -0.02529137 -0.0414103  -0.05263429
  0.0091416  -0.02765045 -0.00656485  0.01953159]
('\nMatrix of word vectors is size : ', (1482L, 121L))


In [6]:
#Create feature vectors from the averages of word vectors

def makeFeatureVec(words, model):
    """
    words - list of words (i.e. article) to be used as input for the creation of word vectors
    model - model to use for the creation of the vectors
    
    makeFeatureVec: Function to average all of the word vectors in a given paragraph
    returns: a numpy array of floats that are the average of the constituent word vectors for each word
    """
    num_features = model.wv.syn0.shape[1]
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the article and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(articles, model):
    """
    articles - list of articles for which the creation of word vectors needs to be done for each
    model - model to use for the creation of the vectors
    
    getAvgFeatureVecs: Given a set of articles (each one a list of words), calculate 
    the average feature vector for each one and return a 2D numpy array 
    returns: a 2D numpy array that contains the average of the constituent word vectors for each article
    """
    num_features = model.wv.syn0.shape[1]
    articleFeatureVecs = np.zeros((len(articles),num_features),dtype="float32")
    counter = 0.
    
    # Loop through the articles
    for article in articles:
        #
        # Print a status message every 1000th review
        if counter%500. == 0.:
            print ("Article %d of %d" % (counter, len(articles)))
        # 
        # Call the function (defined above) that makes average feature vectors
        articleFeatureVecs[counter] = makeFeatureVec(article, model)
        #
        # Increment the counter
        counter = counter + 1.
    return articleFeatureVecs

In [7]:
# Import articles for string cleaning

def clean_text(text, stem_words = False, remove_stopwords = False):
    ###clean
    texts = re.sub("[^a-zA-Z]",    #pattern to match
              " ",              #replace other with this
              str(text))                 #text to apply to
         
    #print "Text recived: ", texts 
    clean_corpus = texts.lower().split()
    #print "corpus: ", clean_corpus
    if stem_words:
        # Porter stemmer
        porter = nltk.PorterStemmer()
        # Snowball stemmer
        snowball = nltk.SnowballStemmer('english')
        # Lancaster stemmer
        lancaster = nltk.LancasterStemmer()
        # General stemming Lambda function to stem tokens
        clean_corpus = lambda tokens: [porter.stem(w) for w in corpus]
    if remove_stopwords:   # Optionally remove stop words
        stops = set(stopwords.words("english"))
        clean_corpus = [w for w in clean_corpus if not w in stops]
        #print "Clean_corpus: ", clean_corpus
    
    return (clean_corpus)

In [8]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. We remove stopwords.

print ("Creating average feature vecs for articles")
cleaned_articles = []
for article in inhibitor.TiAbsMesh:
    cleaned_articles.append(clean_text(article, stem_words=False, remove_stopwords=True ))

averageWordVecs = getAvgFeatureVecs(cleaned_articles, model)

Creating average feature vecs for articles
Article 0 of 671
Article 500 of 671


In [9]:
RANDOM_STATE = 55
TEST_SIZE = 0.3

X_train, X_test, y_train, y_test = train_test_split(averageWordVecs, inhibitor.Label, test_size=TEST_SIZE, 
                                                    random_state=37)

print (len(X_train))
print (len(X_test))

469
202


In [10]:
X = averageWordVecs
y = inhibitor.Label
le = preprocessing.LabelEncoder()
y  = le.fit_transform(y)
yTrFreq = scipy.stats.itemfreq(inhibitor.Label)
print(yTrFreq)

[[  0 647]
 [  1  24]]


In [11]:
def weight(x):
    x = np.where(x==0, 1, 4)
    return x 

In [12]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf', "sigmoid"], 'gamma': [1e-3, 1e-4],
                     'class_weight':['balanced', None],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000], 'class_weight':['balanced', None]}]

scores = ['recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=2, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(metrics.classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for recall
()
Best parameters set found on development set:
()
{'kernel': 'linear', 'C': 100, 'class_weight': 'balanced'}
()
Grid scores on development set:
()
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001, 'class_weight': 'balanced'

In [18]:
clf = SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
              gamma='auto', kernel='linear',max_iter=-1, probability=False, random_state=None, shrinking=True,
              tol=0.001, verbose=False)


seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)
        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

    # print(yTrFreq)
        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])
        
        print ("Fitting for Seed = %d" % seed) 

        clf.fit(X_train, y_train, sample_weight = w)
        pred = clf.predict(X_test)
 
               
        s_vector = clf.n_support_
        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)
        acc_s = metrics.accuracy_score(y_test, pred)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])


        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])

print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[127 197]
 [  0  12]]
Fitting for Seed = 35
[[140 183]
 [  1  11]]
Fitting for Seed = 71
[[146 178]
 [  0  12]]
Fitting for Seed = 71
[[ 97 226]
 [  0  12]]
Fitting for Seed = 21
[[123 201]
 [  0  12]]
Fitting for Seed = 21
[[161 162]
 [  1  11]]
Fitting for Seed = 61
[[139 185]
 [  0  12]]
Fitting for Seed = 61
[[124 199]
 [  0  12]]
Fitting for Seed = 55
[[187 137]
 [  2  10]]
Fitting for Seed = 55
[[110 213]
 [  0  12]]
SVM Cross validation results: 
Mean Precision: 0.06 +/- 0.00 
  Mean  Recall: 0.97 +/- 0.06
      Mean  F1: 0.11 +/- 0.01
Mean  Accuracy: 0.44 +/- 0.07
(' True Negative: ', [127, 140, 146, 97, 123, 161, 139, 124, 187, 110])
('False Negative: ', [0, 1, 0, 0, 0, 1, 0, 0, 2, 0])
(' True Positive: ', [12, 11, 12, 12, 12, 11, 12, 12, 10, 12])
('False Positive: ', [197, 183, 178, 226, 201, 162, 185, 199, 137, 213])
('Negative support Vectors: ', array([ 296.,  283.,  282.,  299.,  297.,  269.,  307.,  277.,  253.,  301.]))
('Positive support Vectors:

In [14]:
clf = SVC(C=10.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
              gamma='auto', kernel='linear', max_iter=-1, probability=False, random_state=55, shrinking=True,
              tol=0.001, verbose=False)

seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)
        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

    # print(yTrFreq)
        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])
        
        print ("Fitting for Seed = %d" % seed) 

        clf.fit(X_train, y_train, sample_weight = w)
        pred = clf.predict(X_test)
 
               
        s_vector = clf.n_support_
        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)
        acc_s = metrics.accuracy_score(y_test, pred)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])


        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])


print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[220 104]
 [  0  12]]
Fitting for Seed = 35
[[203 120]
 [  1  11]]
Fitting for Seed = 71
[[208 116]
 [  2  10]]
Fitting for Seed = 71
[[198 125]
 [  0  12]]
Fitting for Seed = 21
[[236  88]
 [  3   9]]
Fitting for Seed = 21
[[208 115]
 [  1  11]]
Fitting for Seed = 61
[[214 110]
 [  0  12]]
Fitting for Seed = 61
[[203 120]
 [  1  11]]
Fitting for Seed = 55
[[226  98]
 [  3   9]]
Fitting for Seed = 55
[[194 129]
 [  1  11]]
SVM Cross validation results: 
Mean Precision: 0.09 +/- 0.01 
  Mean  Recall: 0.90 +/- 0.09
      Mean  F1: 0.16 +/- 0.01
Mean  Accuracy: 0.66 +/- 0.03
(' True Negative: ', [220, 203, 208, 198, 236, 208, 214, 203, 226, 194])
('False Negative: ', [0, 1, 2, 0, 3, 1, 0, 1, 3, 1])
(' True Positive: ', [12, 11, 10, 12, 9, 11, 12, 11, 9, 11])
('False Positive: ', [104, 120, 116, 125, 88, 115, 110, 120, 98, 129])
('Negative support Vectors: ', array([ 190.,  199.,  195.,  195.,  164.,  186.,  193.,  196.,  156.,  204.]))
('Positive support Vectors: ',

In [20]:
clf = SVC(C=100.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
              gamma='auto', kernel='sigmoid', max_iter=-1, probability=False, random_state=55, shrinking=True,
              tol=0.001, verbose=False)

seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)
        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

    # print(yTrFreq)
        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])
        
        print ("Fitting for Seed = %d" % seed) 

        clf.fit(X_train, y_train, sample_weight = w)
        pred = clf.predict(X_test)
 
               
        s_vector = clf.n_support_
        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)
        acc_s = metrics.accuracy_score(y_test, pred)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])


        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])


print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[114 210]
 [  0  12]]
Fitting for Seed = 35
[[125 198]
 [  0  12]]
Fitting for Seed = 71
[[131 193]
 [  0  12]]
Fitting for Seed = 71
[[ 86 237]
 [  0  12]]
Fitting for Seed = 21
[[112 212]
 [  0  12]]
Fitting for Seed = 21
[[155 168]
 [  0  12]]
Fitting for Seed = 61
[[122 202]
 [  0  12]]
Fitting for Seed = 61
[[105 218]
 [  0  12]]
Fitting for Seed = 55
[[172 152]
 [  2  10]]
Fitting for Seed = 55
[[100 223]
 [  0  12]]
SVM Cross validation results: 
Mean Precision: 0.06 +/- 0.01 
  Mean  Recall: 0.98 +/- 0.05
      Mean  F1: 0.11 +/- 0.01
Mean  Accuracy: 0.40 +/- 0.07
(' True Negative: ', [114, 125, 131, 86, 112, 155, 122, 105, 172, 100])
('False Negative: ', [0, 0, 0, 0, 0, 0, 0, 0, 2, 0])
(' True Positive: ', [12, 12, 12, 12, 12, 12, 12, 12, 10, 12])
('False Positive: ', [210, 198, 193, 237, 212, 168, 202, 218, 152, 223])
('Negative support Vectors: ', array([ 301.,  290.,  288.,  302.,  302.,  274.,  312.,  281.,  264.,  306.]))
('Positive support Vectors:

In [22]:
clf = SVC(C=100.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
              gamma='auto', kernel='rbf', max_iter=-1, probability=False, random_state=55, shrinking=True,
              tol=0.001, verbose=False)

seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)
        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

    # print(yTrFreq)
        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])
        
        print ("Fitting for Seed = %d" % seed) 

        clf.fit(X_train, y_train, sample_weight = w)
        pred = clf.predict(X_test)
 
               
        s_vector = clf.n_support_
        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)
        acc_s = metrics.accuracy_score(y_test, pred)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])


        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])


print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[178 146]
 [  0  12]]
Fitting for Seed = 35
[[163 160]
 [  1  11]]
Fitting for Seed = 71
[[167 157]
 [  0  12]]
Fitting for Seed = 71
[[133 190]
 [  0  12]]
Fitting for Seed = 21
[[174 150]
 [  0  12]]
Fitting for Seed = 21
[[173 150]
 [  1  11]]
Fitting for Seed = 61
[[175 149]
 [  0  12]]
Fitting for Seed = 61
[[141 182]
 [  1  11]]
Fitting for Seed = 55
[[196 128]
 [  3   9]]
Fitting for Seed = 55
[[137 186]
 [  0  12]]
SVM Cross validation results: 
Mean Precision: 0.07 +/- 0.01 
  Mean  Recall: 0.95 +/- 0.08
      Mean  F1: 0.13 +/- 0.01
Mean  Accuracy: 0.52 +/- 0.06
(' True Negative: ', [178, 163, 167, 133, 174, 173, 175, 141, 196, 137])
('False Negative: ', [0, 1, 0, 0, 0, 1, 0, 1, 3, 0])
(' True Positive: ', [12, 11, 12, 12, 12, 11, 12, 11, 9, 12])
('False Positive: ', [146, 160, 157, 190, 150, 150, 149, 182, 128, 186])
('Negative support Vectors: ', array([ 284.,  259.,  259.,  287.,  273.,  251.,  290.,  257.,  225.,  284.]))
('Positive support Vectors: